In [0]:
configs = {
    "fs.azure.account.key.storagedatafiles**.blob.core.windows.net": "*********"
}

dbutils.fs.mount(
    source="wasbs://rawdata@storagedatafiles**.blob.core.windows.net/",
    mount_point="/mnt/rawdata",
    extra_configs=configs
)


True

In [0]:
dbutils.fs.ls("/mnt/rawdata")

[FileInfo(path='dbfs:/mnt/rawdata/transformable_data.csv', name='transformable_data.csv', size=38544, modificationTime=1739870978000)]

In [0]:
df = spark.read.csv("/mnt/rawdata/transformable_data.csv", header=True,inferSchema=True)
df.show()

+---+--------+------+----------+--------+-----------+
| ID|Category| Value|      Date|  Status|Description|
+---+--------+------+----------+--------+-----------+
|  1|       C| 352.1|2022-03-26|Inactive|    Average|
|  2|       D|272.69|2024-01-13|Inactive|    Average|
|  3|       A|161.67|2021-03-29|Inactive|       Good|
|  4|       C|408.76|2021-09-20|Inactive|  Excellent|
|  5|       C|345.52|2023-05-29|  Active|    Average|
|  6|       D| 89.68|2023-03-23|  Active|    Average|
|  7|       A|456.35|2021-10-26|  Active|       Poor|
|  8|       A|413.04|2020-01-12|  Active|       Good|
|  9|       C| 475.4|2023-03-02|Inactive|       Poor|
| 10|       B| 365.6|2020-10-30|  Active|       Poor|
| 11|       C|310.57|2023-08-09|  Active|    Average|
| 12|       C|214.94|2022-09-01|Inactive|       Good|
| 13|       C|467.04|2023-08-16|  Active|  Excellent|
| 14|       C|434.37|2022-02-21|  Active|    Average|
| 15|       D| 32.16|2020-02-12|Inactive|  Excellent|
| 16|       A| 22.92|2022-08

In [0]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import year, month, dayofmonth

df = df.withColumn("Year", year(df["Date"])) \
       .withColumn("Month", month(df["Date"])) \
       .withColumn("Day", dayofmonth(df["Date"]))


In [0]:
df.show()

+---+--------+------+----------+--------+-----------+----+-----+---+
| ID|Category| Value|      Date|  Status|Description|Year|Month|Day|
+---+--------+------+----------+--------+-----------+----+-----+---+
|  1|       C| 352.1|2022-03-26|Inactive|    Average|2022|    3| 26|
|  2|       D|272.69|2024-01-13|Inactive|    Average|2024|    1| 13|
|  3|       A|161.67|2021-03-29|Inactive|       Good|2021|    3| 29|
|  4|       C|408.76|2021-09-20|Inactive|  Excellent|2021|    9| 20|
|  5|       C|345.52|2023-05-29|  Active|    Average|2023|    5| 29|
|  6|       D| 89.68|2023-03-23|  Active|    Average|2023|    3| 23|
|  7|       A|456.35|2021-10-26|  Active|       Poor|2021|   10| 26|
|  8|       A|413.04|2020-01-12|  Active|       Good|2020|    1| 12|
|  9|       C| 475.4|2023-03-02|Inactive|       Poor|2023|    3|  2|
| 10|       B| 365.6|2020-10-30|  Active|       Poor|2020|   10| 30|
| 11|       C|310.57|2023-08-09|  Active|    Average|2023|    8|  9|
| 12|       C|214.94|2022-09-01|In

In [0]:
from pyspark.ml.feature import StringIndexer

category_indexer = StringIndexer(inputCol="Category", outputCol="CategoryIndex")
status_indexer = StringIndexer(inputCol="Status", outputCol="StatusIndex")

df = category_indexer.fit(df).transform(df)
df = status_indexer.fit(df).transform(df)


In [0]:
df.show()

+---+--------+------+----------+--------+-----------+----+-----+---+-------------+-----------+
| ID|Category| Value|      Date|  Status|Description|Year|Month|Day|CategoryIndex|StatusIndex|
+---+--------+------+----------+--------+-----------+----+-----+---+-------------+-----------+
|  1|       C| 352.1|2022-03-26|Inactive|    Average|2022|    3| 26|          2.0|        1.0|
|  2|       D|272.69|2024-01-13|Inactive|    Average|2024|    1| 13|          0.0|        1.0|
|  3|       A|161.67|2021-03-29|Inactive|       Good|2021|    3| 29|          1.0|        1.0|
|  4|       C|408.76|2021-09-20|Inactive|  Excellent|2021|    9| 20|          2.0|        1.0|
|  5|       C|345.52|2023-05-29|  Active|    Average|2023|    5| 29|          2.0|        0.0|
|  6|       D| 89.68|2023-03-23|  Active|    Average|2023|    3| 23|          0.0|        0.0|
|  7|       A|456.35|2021-10-26|  Active|       Poor|2021|   10| 26|          1.0|        0.0|
|  8|       A|413.04|2020-01-12|  Active|       Go

In [0]:
from pyspark.sql.functions import when

df = df.withColumn("Description", 
                   when(df["Description"] == "Excellent", "Very Good")
                   .when(df["Description"] == "Poor", "Needs Improvement")
                   .otherwise(df["Description"]))


In [0]:
df.show()

+---+--------+------+----------+--------+-----------------+----+-----+---+-------------+-----------+
| ID|Category| Value|      Date|  Status|      Description|Year|Month|Day|CategoryIndex|StatusIndex|
+---+--------+------+----------+--------+-----------------+----+-----+---+-------------+-----------+
|  1|       C| 352.1|2022-03-26|Inactive|          Average|2022|    3| 26|          2.0|        1.0|
|  2|       D|272.69|2024-01-13|Inactive|          Average|2024|    1| 13|          0.0|        1.0|
|  3|       A|161.67|2021-03-29|Inactive|             Good|2021|    3| 29|          1.0|        1.0|
|  4|       C|408.76|2021-09-20|Inactive|        Very Good|2021|    9| 20|          2.0|        1.0|
|  5|       C|345.52|2023-05-29|  Active|          Average|2023|    5| 29|          2.0|        0.0|
|  6|       D| 89.68|2023-03-23|  Active|          Average|2023|    3| 23|          0.0|        0.0|
|  7|       A|456.35|2021-10-26|  Active|Needs Improvement|2021|   10| 26|          1.0|   

In [0]:
df.write.mode("append").csv("/mnt/transformeddata")

In [0]:
configs = {
    "fs.azure.account.key.storagedatafiles**.blob.core.windows.net": "*********"
}

dbutils.fs.mount(
    source="wasbs://transformeddata@storagedatafiles**.blob.core.windows.net/",
    mount_point="/mnt/transformeddata",
    extra_configs=configs
)


True

In [0]:
sf_options = {
    "sfURL": "https://****-****.snowflakecomputing.com",
    "sfDatabase": "FIRST_DB",
    "sfSchema": "PUBLIC",
    "sfWarehouse": "COMPUTE_WH",
    "sfRole": "ACCOUNTADMIN",
    "sfUser": "***",
    "sfPassword": "***"
}

In [0]:
df.write.format("snowflake").options(**sf_options).mode("append").option("dbtable", "transformable_data").save()